In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import re
from collections import defaultdict, Counter

class NGramModel:
    def __init__(self, n):
        self.n = n  # Valor de n no modelo N-Grama (e.g., 1 para unigrama, 2 para bigrama)
        self.ngrams = defaultdict(Counter)  # Initializa um defaultdict de Counters para armazenar n-gramas e suas frequencias

    def train(self, text):
        tokens = self._tokenize(text)  # Tokeniza a entrada de texto
        ngrams = self._create_ngrams(tokens)  # Cria n-gramas a partir dos tokens

        for ngram in ngrams:  # Itera sobre as n-gramas criadas
            prefix, token = tuple(ngram[:-1]), ngram[-1]  # Separa a n-grama em prefixo e token
            self.ngrams[prefix][token] += 1  # Incrementa a frequencia do token que sucede o prefixo

    def generate(self, max_words=100):
        output = []
        if self.n == 1:  # Tratando o caso de unigramas
            for _ in range(max_words):
                next_word = self._choose_next(None)  # Unigramas usam None como prefixo
                if next_word is None:
                    break
                output.append(next_word)
        else:
            current = random.choice(list(self.ngrams.keys()))  # Selecione um prefixo inicital aleatoriamente
            output = list(current)  # Inicializa a saída com o prefixo inicial

            for _ in range(max_words - self.n + 1):  # Gera palavras até que max_words seja alcançado
                next_word = self._choose_next(current)  # Escolhe a próxima palavra baseado no prefixo atual
                if next_word is None:  # Se nenhuma palvavra é encontrada, quebre o laço
                    break
                output.append(next_word)  # Adicione a próxima palavra à saída
                current = tuple(output[-self.n+1:])  # Atualize o prefixo atual

        return ' '.join(output)  # Retorna o texto gerado como string

    def _choose_next(self, prefix):
        if self.n == 1:
            options = self.ngrams[()]  # Para unigrama, use uma tupla vazia como chave
        else:
            options = self.ngrams.get(prefix, None)  # Pegue as possíveis próxijmas palavras para o dado prefixo

        if options:  # Se options foi encontrado
            total = sum(options.values())
            probs = [count / total for count in options.values()]  # Calcula as probabilidades
            return random.choices(list(options.keys()), weights=probs)[0]  # Escolha uma palavra baseada nas probabilidades
        else:
            return None  # Retorna None se options não for encontrado

    def _create_ngrams(self, tokens):
        if len(tokens) < self.n:
          return []
        return [tuple(tokens[i:i + self.n]) for i in range(len(tokens) - self.n + 1)]  # Create n-grams from tokens

    def _tokenize(self, text):
        return list(map(str, text.lower()))  # Separa o texto em tokens

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [ ]:
r = requests.get("https://www.gutenberg.org/cache/epub/73984/pg73984-images.html")

r.encoding = 'utf-8'

html = r.text

soup = BeautifulSoup(html, 'html.parser')

text = soup.get_text()

unigram_model = NGramModel(1)
bigram_model = NGramModel(2)
trigram_model = NGramModel(10)
#decgram_model = NGramModel(10)


unigram_model.train(text)
bigram_model.train(text)
trigram_model.train(text)
#decgram_model.train(text)

print("Unigram Generated Text:", unigram_model.generate(max_words=10))
print("Bigram Generated Text:", bigram_model.generate(max_words=10))
print("Trigram Generated Text:", trigram_model.generate(max_words=20))
#rint("decgram Generated Text:", decgram_model.generate(max_words=40))

Unigram Generated Text: d   m u h l d b t  
Bigram Generated Text: - o r e n t o r a r
Trigram Generated Text: t h e   m a i l   r o o m   b e f o r e
